In [ ]:
# importer le drive :
from google.colab import drive
drive.mount('/content/drive')

# Liste des villes disponibles

*   Oulad Barhil
*   Biougra
*   Inezgane
*   Ait Melloul
*   My Drarga
*   Agadir
*   Temsia
*   Oulad Teïma
*   Chichaoua
*   Tameslouht
*   Ait Ourir
*   Marrakech
*   Demnat
*   Errachidia
*   Arfoud
*   Aziylal
*   Al Attawia
*   Youssoufia
*   Safi
*   Ben Guerir
*   Sidi Smai'il
*   Béni Mellal
*   Midalt
*   Settat
*   El Jadid
*   Moulay Abdallah
*   Barrechid
*   Bouskoura
*   Bir Jdid
*   Ad Darwa
*   Casablanca
*   Tit Mellil
*   Aïn Harrouda
*   Mohammedia
*   Beni Yakhlef
*   Al Khmissat
*   Sefrou
*   Oued Zem
*   Azrou
*   El Hajeb
*   Sidi Yahya Zaer
*   Mrirt
*   Skhirate
*   Ain El Aouda
*   Sale
*   Temara
*   Rabat
*   Bouknadel
*   Mehdya
*   Kenitra
*   Meknès
*   Aïn Taoujdat
*   Oulad Tayeb
*   Fès
*   Sidi Qacem
*   Moulay Bousselham
*   Souk et Tnine Jorf el Mellah
*   Taza
*   Guercif
*   Imzouren
*   Taourirt
*   El Aïoun
*   Oujda-Angad
*   Larache
*   Tétouan
*   Tangier
*   Martil
*   M'diq
*   Al Aaroui
*   Al Hoceïma
*   Berkane
*   Nador
*   Iheddadene
*   Zeghanghane

# Evaluation du modèle

In [ ]:
#@title Importations
import torch
import torch.nn as nn
import timm
import pandas as pd
import numpy as np
from PIL import Image
import os
from torchvision import transforms
from sklearn.preprocessing import StandardScaler, LabelEncoder
import matplotlib.pyplot as plt
import seaborn as sns
from tqdm import tqdm
import random


In [ ]:
#@title Configuration du device
device = torch.device("cuda:0" if torch.cuda.is_available() else "cpu")
print(f"Using: {device}")


In [ ]:
#@title Distance Haversine
def haversine_distance(lat1, lon1, lat2, lon2):
    """Calcule la distance Haversine entre deux points"""
    lat1, lon1, lat2, lon2 = map(np.radians, [lat1, lon1, lat2, lon2])
    dlon = lon2 - lon1
    dlat = lat2 - lat1
    a = np.sin(dlat/2)**2 + np.cos(lat1) * np.cos(lat2) * np.sin(dlon/2)**2
    c = 2 * np.arcsin(np.sqrt(a))
    r = 6371  # Rayon de la terre en kilomètres
    return c * r


In [ ]:
#@title Définition du modèle ResNetGPSModelV1
class ResNetGPSModelV1(nn.Module):
    """Modèle ResNet pour prédiction GPS"""
    def __init__(self, num_city_features, optimal_hyperparams):
        super(ResNetGPSModelV1, self).__init__()

        self.resnet = timm.create_model('resnet50', pretrained=True, num_classes=0)

        for name, param in self.resnet.named_parameters():
            if 'layer4' not in name and 'fc' not in name:
                param.requires_grad = False

        resnet_features = 2048

        self.city_features_processor = nn.Sequential(
            nn.Linear(num_city_features, 128),
            nn.BatchNorm1d(128),
            nn.ReLU(),
            nn.Dropout(optimal_hyperparams['dropout_city_1']),
            nn.Linear(128, 256),
            nn.BatchNorm1d(256),
            nn.ReLU(),
            nn.Dropout(optimal_hyperparams['dropout_city_2']),
            nn.Linear(256, 128)
        )

        self.attention = nn.Sequential(
            nn.Linear(resnet_features + 128, 512),
            nn.ReLU(),
            nn.Linear(512, resnet_features + 128),
            nn.Sigmoid()
        )

        self.fc1 = nn.Linear(resnet_features + 128, 1024)
        self.bn1 = nn.BatchNorm1d(1024)
        self.dropout1 = nn.Dropout(optimal_hyperparams['dropout_fc_1'])

        self.fc2 = nn.Linear(1024, 512)
        self.bn2 = nn.BatchNorm1d(512)
        self.dropout2 = nn.Dropout(optimal_hyperparams['dropout_fc_2'])

        self.fc3 = nn.Linear(512, 256)
        self.bn3 = nn.BatchNorm1d(256)
        self.dropout3 = nn.Dropout(optimal_hyperparams['dropout_fc_3'])

        self.residual_connection = nn.Linear(resnet_features + 128, 256)
        self.fc_final = nn.Linear(256, 2)

    def forward(self, image, city_features):
        image_feats = self.resnet(image)
        city_feats = self.city_features_processor(city_features)
        combined = torch.cat((image_feats, city_feats), dim=1)
        attention_weights = self.attention(combined)
        combined = combined * attention_weights

        x = self.fc1(combined)
        x = self.bn1(x)
        x = torch.relu(x)
        x = self.dropout1(x)

        x = self.fc2(x)
        x = self.bn2(x)
        x = torch.relu(x)
        x = self.dropout2(x)

        x = self.fc3(x)
        x = self.bn3(x)
        x = torch.relu(x)
        x = self.dropout3(x)

        residual = self.residual_connection(combined)
        x = x + residual

        coordinates = self.fc_final(x)
        return coordinates


In [ ]:
#@title Préparation des caractéristiques des villes
def prepare_city_features(city_features_path):
    """Prépare les caractéristiques des villes"""
    try:
        city_features = pd.read_csv(city_features_path)
        categorical_cols = ['Architecture', 'Terrain', 'Climat', 'Eau_proche',
                            'Montagne', 'Taille', 'Points_intérêt', 'Grande_ville_proche']
        encoders = {}
        for col in categorical_cols:
            le = LabelEncoder()
            city_features[col] = le.fit_transform(city_features[col])
            encoders[col] = le

        numerical_cols = ['Distance_côte']
        scaler = StandardScaler()
        city_features[numerical_cols] = scaler.fit_transform(city_features[numerical_cols])

        feature_cols = categorical_cols + numerical_cols
        city_features = city_features[['Ville'] + feature_cols + ['lat', 'lng']]

        print(f"Number of cities with features: {len(city_features)}")
        return city_features, feature_cols, encoders, scaler
    except Exception as e:
        print(f"Error loading city features: {e}")
        return None, None, None, None


In [ ]:
#@title Chargement du modèle
def load_model(model_path):
    """Charge le modèle sauvegardé"""
    try:
        checkpoint = torch.load(model_path, map_location=device, weights_only=False)
        num_city_features = checkpoint['num_city_features']
        optimal_hyperparams = checkpoint['optimal_hyperparams']
        model = ResNetGPSModelV1(num_city_features, optimal_hyperparams)
        model.load_state_dict(checkpoint['model_state_dict'])
        model.to(device)
        model.eval()

        print(f"✅ Model loaded successfully from {model_path}")

        return model, optimal_hyperparams
    except Exception as e:
        print(f"❌ Error loading model: {e}")
        return None, None


In [ ]:
#@title Évaluation du modèle
def evaluate_model_on_images(model_path, csv_path, city_features_path, images_folder, n_images=10):
    """
    Évalue le modèle sur n images et affiche les distances entre prédictions et valeurs réelles
    """
    print(f"{'='*80}")
    print(f"MODEL EVALUATION ON {n_images} IMAGES")
    print(f"{'='*80}")

    model, optimal_hyperparams = load_model(model_path)
    if model is None:
        return

    city_features, feature_cols, encoders, scaler = prepare_city_features(city_features_path)
    if city_features is None:
        return

    df_images = pd.read_csv(csv_path)
    print(f"📊 Loaded {len(df_images)} image records from CSV")

    image_files = []
    for root, _, files in os.walk(images_folder):
        for file in files:
            if file.lower().endswith(('.png', '.jpg', '.jpeg')):
                image_files.append(os.path.join(root, file))

    id_to_path = {os.path.splitext(os.path.basename(p))[0]: p for p in image_files}
    print(f"🖼  Found {len(image_files)} images in folder")

    df_images = df_images[df_images['id'].astype(str).isin(id_to_path.keys())]
    df_images = df_images.dropna(subset=['city'])
    df_images = pd.merge(df_images, city_features, left_on='city', right_on='Ville', how='inner')
    df_images = df_images.dropna(subset=feature_cols)

    print(f"📍 {len(df_images)} images have valid city metadata")
    if len(df_images) == 0:
        print("❌ No valid images found for evaluation!")
        return

    if len(df_images) < n_images:
        print(f"⚠  Only {len(df_images)} images available, using all of them")
        n_images = len(df_images)

    selected_images = df_images.sample(n=n_images, random_state=42)

    transform = transforms.Compose([
        transforms.Resize(256),
        transforms.CenterCrop(224),
        transforms.ToTensor(),
        transforms.Normalize([0.485, 0.456, 0.406], [0.229, 0.224, 0.225])
    ])

    results = []
    print(f"\n🔍 Evaluating {n_images} images...")
    print("-" * 80)

    with torch.no_grad():
        for idx, row in tqdm(selected_images.iterrows(), total=n_images, desc="Processing images"):
            try:
                img_id = str(row['id'])
                img_path = id_to_path[img_id]
                image = Image.open(img_path).convert('RGB')
                image_tensor = transform(image).unsqueeze(0).to(device)
                city_feats = row[feature_cols].values.astype(np.float32)
                city_tensor = torch.tensor(city_feats).unsqueeze(0).to(device)
                pred_coords = model(image_tensor, city_tensor).cpu().numpy().flatten()
                true_coords = np.array([row['lat'], row['lng']])
                dist = haversine_distance(pred_coords[0], pred_coords[1], true_coords[0], true_coords[1])
                print(f"[{img_id}] Distance = {dist:.2f} km")
            except Exception as e:
                print(f"Error with image {img_id}: {e}")


In [ ]:
#@title Exécution de l'évaluation
model_path = "/content/drive/MyDrive/model/MarocExplorer_model.pth"  # Remplace par ton chemin
csv_path = "/content/maroc_data_finale.csv"
city_features_path = "/content/drive/MyDrive/Datasets/data_red/M_villes_avec_coords.csv"
images_folder = '/content/drive/MyDrive/Datasets/data_red/iamges_red'

evaluate_model_on_images(
    model_path=model_path,
    csv_path=csv_path,
    city_features_path=city_features_path,
    images_folder=images_folder,
    n_images=10  # Tu peux changer ce nombre
)


# Usage

In [ ]:
#@title Fonction de prédiction pour une seule image
def predict_single_image(image_path, city_name, model_path, city_features_path):
    """
    Prédit la localisation GPS d'une seule image
    """
    SUPPORTED_CITIES = [
        'Oulad Barhil', 'Biougra', 'Inezgane', 'Ait Melloul', 'My Drarga', 'Agadir',
        'Temsia', 'Oulad Teïma', 'Chichaoua', 'Tameslouht', 'Ait Ourir', 'Marrakech',
        'Demnat', 'Errachidia', 'Arfoud', 'Aziylal', 'Al Attawia', 'Youssoufia', 'Safi',
        'Ben Guerir', "Sidi Smai'il", 'Béni Mellal', 'Midalt', 'Settat', 'El Jadid',
        'Moulay Abdallah', 'Barrechid', 'Bouskoura', 'Bir Jdid', 'Ad Darwa',
        'Casablanca', 'Tit Mellil', 'Aïn Harrouda', 'Mohammedia', 'Beni Yakhlef',
        'Al Khmissat', 'Sefrou', 'Oued Zem', 'Azrou', 'El Hajeb', 'Sidi Yahya Zaer',
        'Mrirt', 'Skhirate', 'Ain El Aouda', 'Sale', 'Temara', 'Rabat', 'Bouknadel',
        'Mehdya', 'Kenitra', 'Meknès', 'Aïn Taoujdat', 'Oulad Tayeb', 'Fès',
        'Sidi Qacem', 'Moulay Bousselham', 'Souk et Tnine Jorf el Mellah', 'Taza',
        'Guercif', 'Imzouren', 'Taourirt', 'El Aïoun', 'Oujda-Angad', 'Larache',
        'Tétouan', 'Tangier', 'Martil', "M'diq", 'Al Aaroui', 'Al Hoceïma', 'Berkane',
        'Nador', 'Iheddadene', 'Zeghanghane'
    ]

    print(f"🔍 Predicting GPS location for image: {os.path.basename(image_path)}")
    print(f"📍 City: {city_name}")
    print("-" * 60)

    if city_name not in SUPPORTED_CITIES:
        print(f"❌ Error: '{city_name}' is not in the supported cities list")
        return None

    if not os.path.exists(image_path):
        print(f"❌ Error: Image file not found: {image_path}")
        return None

    try:
        print("⏳ Loading model...")
        model, optimal_hyperparams = load_model(model_path)
        if model is None: return None

        print("📊 Loading city features...")
        city_features, feature_cols, encoders, scaler = prepare_city_features(city_features_path)
        if city_features is None: return None

        city_row = city_features[city_features['Ville'] == city_name]
        if city_row.empty:
            print(f"❌ Error: City '{city_name}' not found in file")
            return None

        print("🖼  Processing image...")
        transform = transforms.Compose([
            transforms.Resize(256),
            transforms.CenterCrop(224),
            transforms.ToTensor(),
            transforms.Normalize([0.485, 0.456, 0.406],
                                 [0.229, 0.224, 0.225])
        ])
        image = Image.open(image_path).convert('RGB')
        image_tensor = transform(image).unsqueeze(0).to(device)

        city_feats = city_row[feature_cols].values[0].astype(np.float32)
        city_tensor = torch.tensor(city_feats).unsqueeze(0).to(device)

        print("🎯 Making prediction...")
        with torch.no_grad():
            pred_coords = model(image_tensor, city_tensor)
            pred_lat, pred_lon = pred_coords.cpu().numpy()[0]

        true_lat = float(city_row['lat'].iloc[0])
        true_lon = float(city_row['lng'].iloc[0])

        distance_to_city_center = haversine_distance(pred_lat, pred_lon, true_lat, true_lon)

        results = {
            'image_path': image_path,
            'city_name': city_name,
            'predicted_latitude': pred_lat,
            'predicted_longitude': pred_lon,
            'city_center_latitude': true_lat,
            'city_center_longitude': true_lon,
            'distance_to_city_center_km': distance_to_city_center
        }

        print("✅ Prediction completed!")
        print(f"🎯 Predicted coordinates: ({pred_lat:.6f}, {pred_lon:.6f})")
        print(f"🏙  City center: ({true_lat:.6f}, {true_lon:.6f})")
        print(f"📏 Distance: {distance_to_city_center:.2f} km")

        return results

    except Exception as e:
        print(f"❌ Error during prediction: {e}")
        return None


In [ ]:
#@title Fonction de visualisation de la prédiction
def predict_and_visualize(image_path, city_name, model_path, city_features_path, save_plot=True):
    """
    Prédit et visualise le résultat sur une carte simple
    """
    results = predict_single_image(image_path, city_name, model_path, city_features_path)
    if results is None: return None

    fig, (ax1, ax2) = plt.subplots(1, 2, figsize=(15, 6))
    image = Image.open(image_path)
    ax1.imshow(image)
    ax1.set_title(f"Input Image\nCity: {city_name}")
    ax1.axis('off')

    pred_lat = results['predicted_latitude']
    pred_lon = results['predicted_longitude']
    city_lat = results['city_center_latitude']
    city_lon = results['city_center_longitude']

    ax2.scatter(pred_lon, pred_lat, color='red', s=100, label='Predicted Location', marker='*')
    ax2.scatter(city_lon, city_lat, color='blue', s=100, label='City Center', marker='o')
    ax2.plot([pred_lon, city_lon], [pred_lat, city_lat], 'k--', alpha=0.5)

    ax2.set_xlabel('Longitude')
    ax2.set_ylabel('Latitude')
    ax2.set_title(f'Distance: {results["distance_to_city_center_km"]:.2f} km')
    ax2.legend()
    ax2.grid(True, alpha=0.3)

    lat_margin = abs(pred_lat - city_lat) * 0.5 + 0.01
    lon_margin = abs(pred_lon - city_lon) * 0.5 + 0.01
    ax2.set_xlim(min(pred_lon, city_lon) - lon_margin, max(pred_lon, city_lon) + lon_margin)
    ax2.set_ylim(min(pred_lat, city_lat) - lat_margin, max(pred_lat, city_lat) + lat_margin)

    plt.tight_layout()
    if save_plot:
        plot_name = f"prediction_{os.path.splitext(os.path.basename(image_path))[0]}.png"
        plt.savefig(plot_name, dpi=300, bbox_inches='tight')
        print(f"📊 Visualization saved: {plot_name}")
    plt.show()
    return results


In [ ]:
#@title Exemple d'exécution pour une image
MODEL_PATH = "/content/drive/MyDrive/model/MarocExplorer_model.pth"
CITY_FEATURES_PATH = "/content/drive/MyDrive/Datasets/data_red/M_villes_avec_coords.csv"

IMAGE_PATH = "/content/tour-hassan-rabat-morocco-by-migel.jpeg"  # Remplacer par votre chemin
CITY_NAME = "Rabat"  # Nom de ville supportée

# Utiliser une des deux fonctions selon le besoin :
# Résultat simple :
results = predict_single_image(
    image_path=IMAGE_PATH,
    city_name=CITY_NAME,
    model_path=MODEL_PATH,
    city_features_path=CITY_FEATURES_PATH
)

# Résultat avec visualisation :
# results = predict_and_visualize(
#     image_path=IMAGE_PATH,
#     city_name=CITY_NAME,
#     model_path=MODEL_PATH,
#     city_features_path=CITY_FEATURES_PATH
# )
